# Load garage datasets

In [1]:
import pandas as pd

df_train_1 = pd.read_parquet("/cnvrg/oblique_cubicasa_train_chips_pj_living_area_dev_v4_chunks/chips/dataset.parquet")
df_train_1["chips_path"] = "/cnvrg/oblique_cubicasa_train_chips_pj_living_area_dev_v4_chunks/chips"
df_train_2 = pd.read_parquet("/cnvrg/oblique_cubicasa_val_chips_pj_living_area_dev_v4/chips/dataset.parquet")
df_train_2["chips_path"] = "/cnvrg/oblique_cubicasa_val_chips_pj_living_area_dev_v4/chips"



In [2]:
df = pd.concat([df_train_2, df_train_1], axis=0)

In [3]:
len(df)

655528

In [4]:
df = df.drop_duplicates(subset=["geometry", 'imagery_source'])

In [5]:
len(df)

655503

In [6]:
val_fraction = 0.1
# Split train, val, test (after grouping by datarow_id!)
from sklearn.model_selection import GroupShuffleSplit 

splitter = GroupShuffleSplit(test_size=val_fraction, n_splits=1, random_state = 7)
split = splitter.split(df, groups=df['cape_full_address'])
train_inds, val_inds = next(split)


X_train = df.iloc[train_inds]
X_val = df.iloc[val_inds]


In [7]:
len(X_train)

589883

In [8]:
len(X_val)

65620

In [9]:
"""import shutil
base_dir = "/cnvrg/dinov2_mtmv_livarea_garages/train"
for label in garage_labels:
    shutil.rmtree(os.path.join(base_dir, label))

base_dir = "/cnvrg/dinov2_mtmv_livarea_garages/val"
for label in garage_labels:
    shutil.rmtree(os.path.join(base_dir, "garages", label))

base_dir = "/cnvrg/dinov2_mtmv_livarea_garages/test"
for label in garage_labels:
    shutil.rmtree(os.path.join(base_dir, "garages", label))
"""

'import shutil\nbase_dir = "/cnvrg/dinov2_mtmv_livarea_garages/train"\nfor label in garage_labels:\n    shutil.rmtree(os.path.join(base_dir, label))\n\nbase_dir = "/cnvrg/dinov2_mtmv_livarea_garages/val"\nfor label in garage_labels:\n    shutil.rmtree(os.path.join(base_dir, "garages", label))\n\nbase_dir = "/cnvrg/dinov2_mtmv_livarea_garages/test"\nfor label in garage_labels:\n    shutil.rmtree(os.path.join(base_dir, "garages", label))\n'

In [10]:
import os
base_dir = "/cnvrg/mtmv_livarea_garages/train"
os.makedirs(os.path.join(base_dir, "livingarea"), exist_ok=True)

base_dir = "/cnvrg/mtmv_livarea_garages/val"
os.makedirs(os.path.join(base_dir, "livingarea"), exist_ok=True)



In [11]:
X = X_val

dataset_path = "/cnvrg/mtmv_livarea_garages/val"

for single_file, chips_dir in zip(X.filename, X.chips_path):
    image_full_path = os.path.join(chips_dir, single_file)
    filename = image_full_path.split("/")[-1]   
    os.symlink(image_full_path, os.path.join(dataset_path, "livingarea", filename))



In [12]:
X = X_train

dataset_path = "/cnvrg/mtmv_livarea_garages/train"

for single_file, chips_dir in zip(X.filename, X.chips_path):
    image_full_path = os.path.join(chips_dir, single_file)
    filename = image_full_path.split("/")[-1]   
    os.symlink(image_full_path, os.path.join(dataset_path, "livingarea", filename))
